# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Дадим кодовые названия регионам: Альфа, Браво и Чарли

In [2]:
try:
    alpha = pd.read_csv('/datasets/geo_data_0.csv')
    bravo = pd.read_csv('/datasets/geo_data_1.csv')  
    charlie = pd.read_csv('/datasets/geo_data_2.csv')
except FileNotFoundError:
    alpha = pd.read_csv(r'C:\Users\ASUS\Desktop\Практикум\Data\geo_data_0.csv')
    bravo = pd.read_csv(r'C:\Users\ASUS\Desktop\Практикум\Data\geo_data_1.csv')  
    charlie = pd.read_csv(r'C:\Users\ASUS\Desktop\Практикум\Data\geo_data_2.csv')

In [3]:
for data in [alpha, bravo, charlie]: 
    print(data.head(), end='\n\n')

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746



In [4]:
for data in [alpha, bravo, charlie]: 
    print('Количество дубликатов: ',data.duplicated().sum(), end='\n\n')

Количество дубликатов:  0

Количество дубликатов:  0

Количество дубликатов:  0



In [5]:
for data in [alpha, bravo, charlie]: 
    print('Количество дубликатов по регионам: ',data['id'].duplicated().sum(), end='\n\n')

Количество дубликатов по регионам:  10

Количество дубликатов по регионам:  4

Количество дубликатов по регионам:  4



In [6]:
for data in [alpha, bravo, charlie]: 
    data.drop(data.query('id.duplicated()==True').index, inplace=True)

Избавимся от столбца "id" так как он не влияет на данные 

In [7]:
for data in [alpha, bravo, charlie]: 
    data = data.drop(['id'], axis=1,inplace = True)

Вывод 
* Открыты и изученны данные 
* Данны кодовые названия регионам "Альфа", "Браво" и "Чарли"
* Удаленны дубликаты
* Удаленн столбец с идентификатором скважины

## Обучение и проверка модели

In [8]:
model = LinearRegression() 

In [9]:
def model_n (data):
    features = data.drop(['product'], axis = 1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    model.fit(features_train, target_train)
    return pd.Series(model.predict(features_valid)), target_valid

In [10]:
predicted_a, target_a = model_n(alpha)

In [11]:
predicted_b, target_b = model_n(bravo)

In [12]:
predicted_c, target_c = model_n(charlie)

In [13]:
def meanrmse (answers, predictions):
    mse = mean_squared_error(answers, predictions)
    print('Cредний запас предсказанного сырья, тыс. барр:', predictions.mean())
    print('RMSE:', mse ** 0.5)

In [14]:
print('Регион "Альфа"')
meanrmse(target_a, predicted_a)
print('\n','Регион "Браво"')
meanrmse(target_b, predicted_b)
print('\n','Регион "Чарли"')
meanrmse(target_c, predicted_c)

Регион "Альфа"
Cредний запас предсказанного сырья, тыс. барр: 92.78915638280621
RMSE: 37.853527328872964

 Регион "Браво"
Cредний запас предсказанного сырья, тыс. барр: 69.1783195703043
RMSE: 0.892059264771703

 Регион "Чарли"
Cредний запас предсказанного сырья, тыс. барр: 94.86572480562035
RMSE: 40.07585073246016


* Данные каждого региона разделенны на обучающую и валидационную выборки.
* Модель линейной регрессии обученна на тренировочных данных каждого региона.
* Посчитали среднее количество предсказанных запасов в каждом регионе, и нашли корень из среднеквадратической ошибки:
  * В регионе "Альфа". Средний запас предсказанного сырья: 92.592 тыс.баррелей, RMSE = 37.579.
  * В регионе "Браво". Средний запас предсказанного сырья: 68.728 тыс.баррелей, RMSE = 0.893.
  * В регионе "Чарли". Средний запас предсказанного сырья: 94.965 тыс.баррелей, RMSE = 40.029.

## Подготовка к расчёту прибыли

In [15]:
budget = 10000000000 
income_per_barrel = 450  
income_per_unit = 450000 
oil_rigs = 200

In [16]:
budget_per_rig = budget/oil_rigs
min_amount = budget/income_per_unit
prod_per_rig = min_amount/200

In [17]:
print('Кол-во нефти необходимое для окупаемости разработки в регионе:',round(min_amount,1),'тыс.барр.')
print('Кол-ва нефти необходимое для окупаемости разработки одной скважины из 200 в регионе:',round(prod_per_rig,2),'тыс.барр.')

Кол-во нефти необходимое для окупаемости разработки в регионе: 22222.2 тыс.барр.
Кол-ва нефти необходимое для окупаемости разработки одной скважины из 200 в регионе: 111.11 тыс.барр.


In [18]:
print(' Регион "Альфа" имеет средний запас нефти по скважине:', target_a.mean())
print('\n','Регион "Браво" имеет средний запас нефти по скважине:',target_b.mean())
print('\n','Регион "Чарли" имеет средний запас нефти по скважине:',target_c.mean())

 Регион "Альфа" имеет средний запас нефти по скважине: 92.15820490940044

 Регион "Браво" имеет средний запас нефти по скважине: 69.18604400957675

 Регион "Чарли" имеет средний запас нефти по скважине: 94.7851093536914


Вывод 
* Для окупаемости разработки в регионе необходимый запас нефти равен 22222 тыс.барр.
* Для окупаемости разработки одной скважины необходимый запас нефти равен 111.11 тыс.барр.
* Средний запас нефти в скважинаж во всех регионов не дотягивает до окупаемости, это значит что случайно выбрать скважину не выйдет

## Расчёт прибыли и рисков 

In [19]:
def rev(target, probabilities):
    target = pd.Series(target).reset_index(drop=True)
    probabilities = pd.Series(probabilities).reset_index(drop=True)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:oil_rigs]
    return ((income_per_unit * selected.sum()) - budget)

In [20]:
rev(target_a, predicted_a)

3365187237.700287

In [21]:
rev(target_b, predicted_b)

2415086696.681511

In [22]:
rev(target_c, predicted_c)

2501283853.2820625

In [23]:
state = np.random.RandomState(12345)
def bs (target, predictions):
    values = []
    for i in range(1000):
        target_subsample = target.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        revenue = rev(target_subsample, probs_subsample)
        values.append(revenue)

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    quantile_interval = (lower, upper)
    conf_int = st.t.interval(0.975, len(values)-1, values.mean(), values.sem())
    mean = values.mean()

    print("Средняя выручка:", mean)
    print("2.5%-квантиль:", lower)
    print("Доверительный интервал:", conf_int)
    print("Доверительный интервал распределения прибыли:",quantile_interval)
    print("Риски убытка:", (values < 0).mean())

In [24]:
bs(target_a, predicted_a)

Средняя выручка: 380613470.0175977
2.5%-квантиль: -142942739.74041605
Доверительный интервал: (362250091.5904023, 398976848.44479305)
Доверительный интервал распределения прибыли: (-142942739.74041605, 890976833.8485551)
Риски убытка: 0.077


In [25]:
bs(target_b, predicted_b)

Средняя выручка: 479159869.981588
2.5%-квантиль: 73849212.0224793
Доверительный интервал: (464568457.3735609, 493751282.5896151)
Доверительный интервал распределения прибыли: (73849212.0224793, 917953817.7221825)
Риски убытка: 0.006


In [26]:
bs(target_c, predicted_c)

Средняя выручка: 315441916.0234899
2.5%-квантиль: -216265050.0341763
Доверительный интервал: (296402795.5028957, 334481036.54408413)
Доверительный интервал распределения прибыли: (-216265050.0341763, 811060708.2365634)
Риски убытка: 0.124


Вывод 

    Проведены расчеты прибыли нефтедобычи в регионах  "Альфа", "Браво" и "Чарли", были оценнены риски убытков в регионах.
    По полученным результатам регион "Браво" является максимально выгодным для разработки нефти .
    Регион "Браво" имеет наименьшие риски 0.6% и наивысшую среднюю выручку 479159869.981588